In [2]:
import sys
sys.path.append('/home/pi/MasterPi/')
import time
import signal
import HiwonderSDK.mecanum as mecanum
import ipywidgets.widgets as widgets
from IPython.display import display

In [3]:
chassis = mecanum.MecanumChassis()

In [4]:
chassis.set_velocity(0,0,0)

In [5]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
leftturn_button = widgets.Button(description='left Turn', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
rightturn_button = widgets.Button(description='right turn', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
left_box = widgets.HBox([leftturn_button, forward_button, rightturn_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([left_box, middle_box, backward_button])
display(controls_box)

In [6]:
def stop(change):
    chassis.set_velocity(0,0,0)
    
def step_forward(change):
    chassis.set_velocity(50,90,0)

def step_backward(change):
    chassis.set_velocity(50,270,0)

def step_left(change):
    chassis.set_velocity(50,180,0)

def step_right(change):
    chassis.set_velocity(50,0,0)

def step_leftturn(change):
    chassis.set_velocity(0,90,-0.3)# rotate counterclockwise

def step_rightturn(change):
    chassis.set_velocity(0,90,0.3)# rotate clockwise


In [7]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
leftturn_button.on_click(step_leftturn)
rightturn_button.on_click(step_rightturn)

In [8]:
import threading
import yaml_handle
from ArmIK.Transform import *
from ArmIK.ArmMoveIK import *
import HiwonderSDK.Board as Board

In [9]:
AK = ArmIK()
global lab_data, servo_data
servo1 = 1500
def initMove():
    Board.setPWMServoPulse(1, servo1, 800)
    AK.setPitchRangeMoving((0, 8, 10), -90, -90, 0, 1500)


In [10]:
rect = None
size = (640, 480)
rotation_angle = 0
unreachable = False 
world_X, world_Y = 0, 0
global rect
global _stop
global get_roi
global unreachable
global __isRunning
global detect_color
global start_pick_up
global rotation_angle
global world_X, world_Y

#placement coordinate
coordinate = {
    'red':   (-15, 14, 2),
    'green': (-18, 9,  3),
    'blue':  (-18, 0, 2),
    'capture': (0, 16.5, 2)
}


In [22]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [25]:
def f(x):
    return x

In [24]:
interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

In [10]:
detect_color = 'green'

In [12]:

AK.setPitchRangeMoving((0, 6, 18), 0,-90, 90, 1500) # lift up robotic arm 
time.sleep(1.5)
Board.setPWMServoPulse(1, 2000, 500) # the gripper opens
time.sleep(1.5)
Board.setPWMServoPulse(1, 1500, 500) # the gripper closes
time.sleep(1.5)
if detect_color == 'red':       # The robotic arm rotates to the corresponding angle according to the detected color.
    Board.setPWMServoPulse(6, 1900, 500)
    time.sleep(0.5)
elif detect_color == 'green':
    Board.setPWMServoPulse(6, 2100, 800)
    time.sleep(0.8)
elif detect_color == 'blue':
    Board.setPWMServoPulse(6, 2500, 1500)
    time.sleep(1.5)
result = AK.setPitchRangeMoving((coordinate[detect_color][0], coordinate[detect_color][1], 8), -90, -90, 0) # Move to above the coordinates of the corresponding color
if result == False:
    unreachable = True
else:
    unreachable = False
    time.sleep(result[2]/1000) #If it can get to the specific posiiton, the running time is obtained.
AK.setPitchRangeMoving((coordinate[detect_color]), -90, -90, 0, 500)  # Place at the coordinate of the detected color
time.sleep(0.5)
Board.setPWMServoPulse(1, 1800, 500) # The gripper opens 
time.sleep(0.5)
AK.setPitchRangeMoving((coordinate[detect_color][0], coordinate[detect_color][1], 8), -90, -90, 0, 800) #  Move to above the coordinates of the corresponding color
time.sleep(0.8)
Board.setPWMServosPulse([1200, 4, 1,1500, 3,515, 4,2170, 5,945]) # reset the robotic arm
time.sleep(1.2)
if detect_color == 'red':
    Board.setPWMServoPulse(6, 1500, 500)
    time.sleep(0.5)
elif detect_color == 'green':
    Board.setPWMServoPulse(6, 1500, 800)
    time.sleep(0.8)
elif detect_color == 'blue':
    Board.setPWMServoPulse(6, 1500, 1500)
    time.sleep(1.5)
AK.setPitchRangeMoving((0, 8, 10), -90, -90, 0, 1000)
time.sleep(1)
                
